#Table of Contents

1. Import Libraries
2. Load Dataset
3. Create Field Objects
4. Data Preparation
  - Build Vocabulary
  - Create Dataloaders
  
5. Define Model Architecture
  - Encoder Architecture
  - Attention Mechanism
  - Decoder Architecture
  - Sequence-to-Sequence Architecture
7. Train Sequence-to-Sequence Model
8. Model Inference 
  - Build Inference Function
  - Translate Russian Sentences in the Test Dataset

#1. Import Libraries

In [3]:
import re
import time
import math
import random

import numpy as np
import pandas as pd
import spacy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data

from tqdm import notebook
pd.set_option('display.max_colwidth', 200)

In [4]:
# check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#2. Load Dataset

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# extract the zip file from your Google Drive
! unzip '/content/drive/My Drive/Course_Notes/NLP using PyTorch/Seq2Seq/nmt_data.zip'

#3. Create Field Objects

In [5]:
# import Russian spacy model to tokenize Russian text
from spacy.lang.ru import Russian

In [6]:
# dependency for spaCy Russian tokenizer
!pip install pymorphy2

In [7]:
# spacy object for Russian
nlp_ru = Russian()

# spacy object for English
nlp_en = spacy.load("en_core_web_sm", disable = ["parser", "tagger", "ner"])

In [8]:
## functions to perform tokenization

# tokenizes Russian text from a string into a list of tokens
def tokenize_ru(text):
  return [tok.text for tok in nlp_ru.tokenizer(text)]

# tokenizes English text from a string into a list of tokens
def tokenize_en(text):
  return [tok.text for tok in nlp_en.tokenizer(text)]

In [9]:
## Create Field objects

# Field object for Russian
SRC = data.Field(tokenize = tokenize_ru, 
                 include_lengths = True, 
                 lower = True)

# Field object for English
TRG = data.Field(tokenize = tokenize_en, 
                 init_token = '<sos>', # "start" token
                 eos_token = '<eos>', # "" token
                 include_lengths = True, 
                 lower = True)

fields = [('rus', SRC), ('eng', TRG)]

* refer the video "Text preprocessing in PyTorch" in the course "Fundamentals of Deep Learning" to learn more about the TorchText's Field objects

#4. Data Preparation

###4.1 Build Vocabulary

In [15]:
# importing data from csv
nmt_data = data.TabularDataset(path="nmt_data.csv", format='csv', fields=fields)

In [16]:
# build vocabulary for Russian sequences
SRC.build_vocab(nmt_data, max_size=4000)

# build vocabulary for English sequences
TRG.build_vocab(nmt_data, max_size=4000)

In [17]:
# check size of vocabulary
len(SRC.vocab), len(TRG.vocab)

(4002, 4004)

###4.2 Create Dataloaders

In [18]:
# Split our dialogue data into training, validation, and test sets
train_data, val_data = nmt_data.split(split_ratio=0.8)

In [19]:
# Create a set of iterators for each split
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, val_data), 
    batch_size = 64, 
    sort_within_batch = True, 
    sort_key = lambda x:len(x.rus),
    device = device)

#5. Define Model Architecture

###5.1 Encoder Architecture

In [10]:
## embedding layer: 
##    input dimensions = size of Russian vocabulary
##    ouput dimensions = embedding_size

## GRU layer:
##    input dimensions = embedding_size
##    hidden units = hidden_size
##    layers = num_layers
##    output dim = hidden_size

class Encoder(nn.Module):
  
  def __init__(self, hidden_size, embedding_size, num_layers=2, dropout=0.3):
    
    super(Encoder, self).__init__()
    
    # Basic network params
    self.hidden_size = hidden_size
    self.embedding_size = embedding_size
    self.num_layers = num_layers
    self.dropout = dropout
    
    # Embedding layer that will be shared with Decoder
    self.embedding = nn.Embedding(len(SRC.vocab), embedding_size)
    # GRU layer
    self.gru = nn.GRU(embedding_size, hidden_size,
                      num_layers=num_layers,
                      dropout=dropout)
      
  def forward(self, input_sequence):
      
    # Convert input_sequence to word embeddings
    embedded = self.embedding(input_sequence)
            
    outputs, hidden = self.gru(embedded)
    
    # The ouput of a GRU has shape -> (seq_len, batch, hidden_size)
    return outputs, hidden

###5.2 Attention Mechanism

In [11]:
class Attention(nn.Module):
  def __init__(self, hidden_size):
    super(Attention, self).__init__()        
    self.hidden_size = hidden_size
      
    
  def dot_score(self, hidden_state, encoder_states):
    return torch.sum(hidden_state * encoder_states, dim=2)
  
          
  def forward(self, hidden, encoder_outputs, mask):
      
    attn_scores = self.dot_score(hidden, encoder_outputs)
    
    # Transpose max_length and batch_size dimensions
    attn_scores = attn_scores.t()
    
    # Apply mask so network does not attend <pad> tokens        
    attn_scores = attn_scores.masked_fill(mask == 0, -1e5)
    
    # Return softmax over attention scores      
    return F.softmax(attn_scores, dim=1).unsqueeze(1)

###5.3 Decoder Architecture

In [12]:
## embedding layer: 
##    input dimensions = output_size (size of English vocabulary), 
##    ouput dimensions = embedding_size

## GRU layer:
##    input dimensions = embedding_size
##    hidden units = hidden_size
##    layers = n_layers
##    output dim = hidden_size

## concat layer:
##    input dimensions = hidden_size * 2
##    output dimensions = hidden_size

## fully Connected layer:
##    input dimensions = hidden_size, 
##    ouput dimensions = output_size (size of English vocabulary)

class Decoder(nn.Module):
  def __init__(self, embedding_size, hidden_size, output_size, n_layers=2, dropout=0.3):
      
    super(Decoder, self).__init__()
    
    # Basic network params
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    self.dropout = dropout
    self.embedding = nn.Embedding(output_size, embedding_size)
            
    self.gru = nn.GRU(embedding_size, hidden_size, n_layers, 
                      dropout=dropout)
    
    self.concat = nn.Linear(hidden_size * 2, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.attn = Attention(hidden_size)
      
  def forward(self, current_token, hidden_state, encoder_outputs, mask):
    
    # convert current_token to word_embedding
    embedded = self.embedding(current_token)
    
    # Pass through GRU
    gru_output, hidden_state = self.gru(embedded, hidden_state)
    
    # Calculate attention weights
    attention_weights = self.attn(gru_output, encoder_outputs, mask)
    
    # Calculate context vector (weigthed average)
    context = attention_weights.bmm(encoder_outputs.transpose(0, 1))
    
    # Concatenate  context vector and GRU output
    gru_output = gru_output.squeeze(0)
    context = context.squeeze(1)
    concat_input = torch.cat((gru_output, context), 1)
    concat_output = torch.tanh(self.concat(concat_input))
    
    # Pass concat_output to final output layer
    output = self.out(concat_output)
    
    # Return output and final hidden state
    return output, hidden_state

###5.4 Sequence-to-Sequence Architecture

In [13]:
class seq2seq(nn.Module):
  def __init__(self, embedding_size, hidden_size, vocab_size, device, pad_idx, eos_idx, sos_idx):
    super(seq2seq, self).__init__()
    
    # Embedding layer shared by encoder and decoder
    self.embedding = nn.Embedding(vocab_size, embedding_size)
    
    # Encoder network
    self.encoder = Encoder(hidden_size, 
                            embedding_size,
                            num_layers=2,
                            dropout=0.3)
    
    # Decoder network        
    self.decoder = Decoder(embedding_size,
                            hidden_size,
                            vocab_size,
                            n_layers=2,
                            dropout=0.3)
    
    
    # Indices of special tokens and hardware device 
    self.pad_idx = pad_idx
    self.eos_idx = eos_idx
    self.sos_idx = sos_idx
    self.device = device
      
  def create_mask(self, input_sequence):
    return (input_sequence != self.pad_idx).permute(1, 0)
      
      
  def forward(self, input_sequence, output_sequence):
    
    # Unpack input_sequence tuple
    input_tokens = input_sequence[0]
  
    # Unpack output_tokens, or create an empty tensor for text generation
    if output_sequence is None:
      inference = True
      output_tokens = torch.zeros((100, input_tokens.shape[1])).long().fill_(self.sos_idx).to(self.device)
    else:
      inference = False
      output_tokens = output_sequence[0]
    
    vocab_size = self.decoder.output_size
    batch_size = len(input_sequence[1])
    max_seq_len = len(output_tokens)
    
    # tensor to store decoder outputs
    outputs = torch.zeros(max_seq_len, batch_size, vocab_size).to(self.device)        
    
    # pass input sequence to the encoder
    encoder_outputs, hidden = self.encoder(input_tokens)
    
    # first input to the decoder is the <sos> tokens
    output = output_tokens[0,:]
    
    # create mask
    mask = self.create_mask(input_tokens)
    
    
    # Step through the length of the output sequence one token at a time
    for t in range(1, max_seq_len):
      output = output.unsqueeze(0)
      
      output, hidden = self.decoder(output, hidden, encoder_outputs, mask)
      outputs[t] = output
      
      if inference:
        output = output.max(1)[1]
      else:
        output = output_tokens[t]
      
      # If we're in inference mode, keep generating until we produce an
      # <eos> token
      if inference and output.item() == self.eos_idx:
        return outputs[:t]
        
    return outputs

#6. Train Seq2Seq Model

In [20]:
# extract special tokens
pad_idx = TRG.vocab.stoi['<pad>']
eos_idx = TRG.vocab.stoi['<eos>']
sos_idx = TRG.vocab.stoi['<sos>']

# Size of embedding_dim should match the dim of pre-trained word embeddings!
embedding_dim = 100
hidden_dim = 256
vocab_size = len(TRG.vocab)

In [21]:
model = seq2seq(embedding_dim,
                hidden_dim, 
                vocab_size, 
                device, pad_idx, eos_idx, sos_idx).to(device)

In [22]:
# print model architecture
model

seq2seq(
  (embedding): Embedding(4004, 100)
  (encoder): Encoder(
    (embedding): Embedding(4002, 100)
    (gru): GRU(100, 256, num_layers=2, dropout=0.3)
  )
  (decoder): Decoder(
    (embedding): Embedding(4004, 100)
    (gru): GRU(100, 256, num_layers=2, dropout=0.3)
    (concat): Linear(in_features=512, out_features=256, bias=True)
    (out): Linear(in_features=256, out_features=4004, bias=True)
    (attn): Attention()
  )
)

In [ ]:
# Adam optimizer
optimizer = optim.Adam(model.parameters())

# cross entropy loss with softmax
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

In [ ]:
def train(model, iterator, criterion, optimizer):
  # Put the model in training mode!
  model.train()
  
  epoch_loss = 0
  
  for idx, batch in notebook.tqdm(enumerate(iterator), total=len(iterator)):
    input_sequence = batch.rus
    output_sequence = batch.eng

    target_tokens = output_sequence[0]

    # zero out the gradient for the current batch
    optimizer.zero_grad()

    # Run the batch through our model
    output = model(input_sequence, output_sequence)

    # Throw it through our loss function
    output = output[1:].view(-1, output.shape[-1])
    target_tokens = target_tokens[1:].view(-1)

    loss = criterion(output, target_tokens)

    # Perform back-prop and calculate the gradient of our loss function
    loss.backward()

    # Update model parameters
    optimizer.step()

    epoch_loss += loss.item()
      
  return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
  # Put the model in training mode!
  model.eval()
  
  epoch_loss = 0
  
  for idx, batch in notebook.tqdm(enumerate(iterator), total=len(iterator)):
    input_sequence = batch.rus
    output_sequence = batch.eng

    target_tokens = output_sequence[0]

    # Run the batch through our model
    output = model(input_sequence, output_sequence)

    # Throw it through our loss function
    output = output[1:].view(-1, output.shape[-1])
    target_tokens = target_tokens[1:].view(-1)

    loss = criterion(output, target_tokens)

    epoch_loss += loss.item()
      
  return epoch_loss / len(iterator)

In [ ]:
# function to compute time taken by an epoch (in mm:ss)
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

# start model training
for epoch in range(N_EPOCHS):
    
  start_time = time.time()
  
  train_loss = train(model, train_iterator, criterion, optimizer)
  valid_loss = evaluate(model, valid_iterator, criterion)
  
  end_time = time.time()
  
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  
  # compare validation loss
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'best_model.pt')
  
  print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


Epoch: 01 | Time: 1m 12s
	Train Loss: 3.030 | Train PPL:  20.690
	 Val. Loss: 2.123 |  Val. PPL:   8.357



Epoch: 02 | Time: 1m 12s
	Train Loss: 1.858 | Train PPL:   6.412
	 Val. Loss: 1.632 |  Val. PPL:   5.112



Epoch: 03 | Time: 1m 10s
	Train Loss: 1.492 | Train PPL:   4.447
	 Val. Loss: 1.446 |  Val. PPL:   4.245



Epoch: 04 | Time: 1m 10s
	Train Loss: 1.308 | Train PPL:   3.699
	 Val. Loss: 1.349 |  Val. PPL:   3.852



Epoch: 05 | Time: 1m 10s
	Train Loss: 1.196 | Train PPL:   3.307
	 Val. Loss: 1.300 |  Val. PPL:   3.669



Epoch: 06 | Time: 1m 11s
	Train Loss: 1.118 | Train PPL:   3.059
	 Val. Loss: 1.269 |  Val. PPL:   3.558



Epoch: 07 | Time: 1m 10s
	Train Loss: 1.060 | Train PPL:   2.887
	 Val. Loss: 1.246 |  Val. PPL:   3.476



Epoch: 08 | Time: 1m 10s
	Train Loss: 1.013 | Train PPL:   2.755
	 Val. Loss: 1.231 |  Val. PPL:   3.424



Epoch: 09 | Time: 1m 10s
	Train Loss: 0.976 | Train PPL:   2.654
	 Val. Loss: 1.224 |  Val. PPL:   3.401



Epoch: 10 | Time: 1m 11s
	Train Loss: 0.944 | Train PPL:   2.569
	 Val. Loss: 1.216 |  Val. PPL:   3.373


#7. Model Inference

In [23]:
# load saved model weights
path = 'best_model.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

###7.1 Build Inference Function

In [24]:
def translate_sentence(model, sentence):
    model.eval()
    
    # tokenization
    tokenized = nlp_ru(sentence) 
    # convert tokens to lowercase
    tokenized = [t.lower_ for t in tokenized]
    # convert tokens to integers
    int_tokenized = [SRC.vocab.stoi[t] for t in tokenized] 
    
    # convert list to tensor
    sentence_length = torch.LongTensor([len(int_tokenized)]).to(model.device) 
    tensor = torch.LongTensor(int_tokenized).unsqueeze(1).to(model.device) 
    
    # get predictions
    translation_tensor_logits = model((tensor, sentence_length), None) 
    
    # get token index with highest score
    translation_tensor = torch.argmax(translation_tensor_logits.squeeze(1), 1)
    # convert indices (integers) to tokens
    translation = [TRG.vocab.itos[t] for t in translation_tensor]
 
    # Start at the first index.  We don't need to return the <sos> token...
    translation = translation[1:]
    return " ".join(translation)

In [ ]:
sentence = "это новый"
response = translate_sentence(model, sentence)
print(response)

is that a new


###7.2 Translate Russian Sentences in the Test Dataset

In [27]:
# read test file 
test_df = pd.read_csv('nmt_test_translations.csv')

In [28]:
# attention based translations
attn_translations = [translate_sentence(model, sent) for sent in notebook.tqdm(test_df["rus"])]

In [29]:
test_df["attn_translations"] = attn_translations

In [30]:
# check translations
test_df.sample(20)

,rus,eng,translations,attn_translations
23503,том прислал мне подарок,tom sent me a present,tom sent me a gift,tom sent me a present
9567,ты им не понравилась,they didn't like you,did n't you like them,they did n't like you
23039,здесь часто идёт снег,it often snows here,it 's snowing here,it 's snowing here often
10078,он находится в розыске,he's wanted by the police,he 's in the <unk>,he 's in the <unk>
43445,они нас обманули,they deceived us,they 're going to kill us,they 're going to kill us
31151,кто тебе дал этот кошелёк,who gave you this wallet,who gave you this photo,who gave you that necklace
46255,я знаю что это правда,i know that that's true,i know that 's true,i know it 's true
41621,не смейтесь надо мной,don't make fun of me,do n't laugh at me,do n't make fun of me
420,это для меня так важно,it's so important to me,it 's important to me that,that 's so important to me
37908,я попросил тома нам помочь,i asked tom to help us,i asked tom to help us,i asked tom to help us
